In [1]:
import pandas as pd
from src.db import get_dataframe, save_to_duckdb

# Add description here
#
# *Note:* You can open this file as a notebook (JupyterLab: right-click on it in the side bar -> Open With -> Notebook)

In [2]:
# Uncomment the next two lines to enable auto reloading for imported modules
# %load_ext autoreload
# %autoreload 2
# For more info, see:
# https://docs.ploomber.io/en/latest/user-guide/faq_index.html#auto-reloading-code-in-jupyter

In [3]:
# If this task has dependencies, list them them here
# (e.g. upstream = ['some_task']), otherwise leave as None.
upstream = ['s01_get']

# This is a placeholder, leave it as None
product = None
table_name = None
transformed_table_name = None

In [4]:
# Parameters
table_name = "house_prices"
transformed_table_name = "house_prices_transformed"
upstream = {"s01_get": {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/products/01-extract-pipeline.ipynb", "csv_file": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/data/house_prices_raw.csv", "database": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/data/house_prices.duckdb"}}
product = {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/house_pricing/products/04-transform.ipynb"}


In [5]:
# your code here...
db_file = upstream['s01_get']['database']
df = get_dataframe(duckdb_file=db_file, table_name=table_name)

In [6]:
df.shape

(20640, 14)

In [7]:
X = df.drop(['median_house_value'], axis=1)
y = df['median_house_value']

In [8]:
from sklearn.preprocessing import StandardScaler

features = X.columns  # ["col1", "col2", "col3", "col4"]
autoscaler = StandardScaler()
X[features] = autoscaler.fit_transform(X[features])

In [9]:
# --Variance Inflation Factor
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_df = pd.DataFrame()
vif_x = X.copy()
vif_df["feature"] = vif_x.columns
vif_df["VIF"] = [variance_inflation_factor(vif_x.values, i)
                 for i in range(len(vif_x.columns))]

vif_df.head(10)

,feature,VIF
0,median_income,1.797414
1,median_age,1.406807
2,tot_rooms,12.936392
3,tot_bedrooms,36.650481
4,population,6.502235
5,households,35.831425
6,latitude,157.362679
7,longitude,49.879352
8,distance_to_coast,4.136801
9,distance_to_la,15.387614


In [10]:
X = X.join(y)

In [11]:
X.shape

(20640, 14)

In [12]:
X.describe()

,median_income,median_age,tot_rooms,tot_bedrooms,population,households,latitude,longitude,distance_to_coast,distance_to_la,distance_to_sandiego,distance_to_sanjose,distance_to_sanfrancisco,median_house_value
count,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,2.064000e+04,20640.000000
mean,6.609700e-17,5.508083e-18,3.201573e-17,3.304850e-17,-1.101617e-17,6.885104e-17,-1.079584e-15,-8.526513e-15,2.203233e-17,2.203233e-17,4.406467e-17,3.525173e-16,8.812933e-17,206855.816909
std,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,1.000024e+00,115395.615874
min,-1.774299e+00,-2.196180e+00,-1.207283e+00,-1.274573e+00,-1.256123e+00,-1.303984e+00,-1.447568e+00,-2.385992e+00,-8.219279e-01,-1.085881e+00,-1.374184e+00,-1.605465e+00,-1.544212e+00,14999.000000
25%,-6.881186e-01,-8.453931e-01,-5.445698e-01,-5.766293e-01,-5.638089e-01,-5.742294e-01,-7.967887e-01,-1.113209e+00,-6.396061e-01,-9.579548e-01,-8.249615e-01,-1.087145e+00,-1.076672e+00,119600.000000
50%,-1.767951e-01,2.864572e-02,-2.332104e-01,-2.442754e-01,-2.291318e-01,-2.368162e-01,-6.422871e-01,5.389137e-01,-4.067504e-01,-3.865333e-01,-6.338258e-01,5.092059e-01,5.591732e-01,179700.000000
75%,4.593063e-01,6.643103e-01,2.348028e-01,2.590034e-01,2.644949e-01,2.758427e-01,9.729566e-01,7.784964e-01,1.896900e-01,1.040399e+00,1.063018e+00,7.725678e-01,7.910869e-01,264725.000000
max,5.858286e+00,1.856182e+00,1.681558e+01,1.402320e+01,3.025033e+01,1.460152e+01,2.958068e+00,2.625280e+00,5.968708e+00,3.022843e+00,2.760097e+00,2.245394e+00,2.066797e+00,500001.000000


In [13]:
save_to_duckdb(df=X, table_name=transformed_table_name, db_path=upstream['s01_get']['database'])

In [14]:
import duckdb

conn = duckdb.connect(upstream['s01_get']['database'])
conn.sql('SHOW TABLES;')

┌──────────────────────────┐
│           name           │
│         varchar          │
├──────────────────────────┤
│ house_prices             │
│ house_prices_transformed │
└──────────────────────────┘

In [15]:

conn.close()